In [2]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import math
import jieba
from collections import defaultdict
from tqdm import tqdm
import time
import os
import glob

In [21]:
task_list = glob.glob('/mnt/disk1/jiaoyimin/fenlei/*.txt')

In [22]:
for txt_fn in task_list:
    exist_file = os.listdir('/mnt/disk1/jiaoyimin/result_1')
    for exist_fn in exist_file:
        if exist_fn in txt_fn:
            task_list.remove(txt_fn)

In [23]:
len(exist_file)

0

In [24]:
len(task_list)

93

In [25]:
task_list # 医药卫生总论.txt 太大了，可能内存不够，不行的话暂时跳过

['/mnt/disk1/jiaoyimin/fenlei/中国医学.txt',
 '/mnt/disk1/jiaoyimin/fenlei/大学学报(工业技术).txt',
 '/mnt/disk1/jiaoyimin/fenlei/军事科技.txt',
 '/mnt/disk1/jiaoyimin/fenlei/党建.txt',
 '/mnt/disk1/jiaoyimin/fenlei/科研管理.txt',
 '/mnt/disk1/jiaoyimin/fenlei/地理.txt',
 '/mnt/disk1/jiaoyimin/fenlei/经济与管理.txt',
 '/mnt/disk1/jiaoyimin/fenlei/电工技术.txt',
 '/mnt/disk1/jiaoyimin/fenlei/劳动与人才.txt',
 '/mnt/disk1/jiaoyimin/fenlei/力学.txt',
 '/mnt/disk1/jiaoyimin/fenlei/矿业工程.txt',
 '/mnt/disk1/jiaoyimin/fenlei/历史.txt',
 '/mnt/disk1/jiaoyimin/fenlei/工业经济.txt',
 '/mnt/disk1/jiaoyimin/fenlei/林业.txt',
 '/mnt/disk1/jiaoyimin/fenlei/动力工程.txt',
 '/mnt/disk1/jiaoyimin/fenlei/逻辑伦理.txt',
 '/mnt/disk1/jiaoyimin/fenlei/预防医学与卫生学.txt',
 '/mnt/disk1/jiaoyimin/fenlei/马列主义理论.txt',
 '/mnt/disk1/jiaoyimin/fenlei/法律.txt',
 '/mnt/disk1/jiaoyimin/fenlei/贸易经济.txt',
 '/mnt/disk1/jiaoyimin/fenlei/妇产科学与儿科学.txt',
 '/mnt/disk1/jiaoyimin/fenlei/内科学.txt',
 '/mnt/disk1/jiaoyimin/fenlei/农学.txt',
 '/mnt/disk1/jiaoyimin/fenlei/化学.txt',
 '/mnt/disk1/ji

In [26]:
            
def load_data(filename, stopwords):

    data = []
    with open(filename, 'r') as f:
        for line in f:
            word_list = [x for x in jieba.cut(line.strip(), cut_all=False) if x not in stopwords]
            data.append(word_list)
    return data

 
    
def info_entropy(array):
    array = np.asarray(array)
    return np.sum(-(array / array.sum() ) * (np.log(array / array.sum())))    

def generate_ntuple(input_list, n):
    result = []
    for i in range(1, n+1):
        result.extend(zip(*[input_list[j:] for j in range(i)]))
    return result

In [27]:
stopwords = '''+-×:!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…'''  

In [28]:
stopwords1 = open('chinesestopwords.txt', 'r',encoding = 'utf-8')

In [29]:
stopwords1= stopwords1.read().splitlines()

In [5]:
#检验是否含中文字符
def ContainChinese(s):
    for ch in s:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

In [31]:

for txt_fn in tqdm(task_list):
    print(txt_fn,'Cut words')
    #file = open(txt_fn,'r')
    fW = load_data(txt_fn,stopwords)
    #for line in file:
        #word_list = [x for x in jieba.cut(line.strip(), cut_all=False) if x not in stopwords]     
        #tuples4 = generate_ntuple(word_list, 4)
    print(txt_fn,'Done Cut')
    for jieba_content in fW:
        tuples4 = generate_ntuple(jieba_content, 4)
        tuple4 = dict(Counter(list(tuples4)))
    
        all_2tuple = dict()
        all_3tuple = dict()
        all_1tuple = dict()
        all_4tuple = dict()
        for key,value in tuple4.items():
            if len(key)==2:
                all_2tuple[key]=value
            #if len(key)==3:
                all_3tuple[key]=value
            if len(key)==1:
                all_1tuple[key]=value  
            if len(key)==4:
                all_4tuple[key]=value
    
    
    ###all_tuples.append(all_2tuple)
    #all_tuples.append(all_3tuple)
    #all_tuples.append(all_4tuple)
        total2 = len(all_2tuple)
    
        result = defaultdict(dict)
    #pmi = dict()


        for word in all_2tuple:
            left0 = word[:1]
            left_count0 = all_1tuple[left0]
            right0 = word[1:]
            right_count0 = all_1tuple[right0]
            a0 = math.log(total2*all_2tuple[word] / left_count0 / right_count0)
            
        #remain.append(word)
            result[word]['pmi'] = a0
            result[word]['ami'] = a0/(len(word[0])+len(word[1]))
            result[word]['freq'] = all_2tuple[word]
        #pmi[word] = a0

    
        final_words = []

        mutual_info = {}


        for word in all_4tuple:
            left = word[0]
            right = word[3]
            tup = word[1:3]
            if tup not in mutual_info:
                mutual_info[tup] = {'left':{},'right':{}}
            if left not in mutual_info[tup]['left']:
                mutual_info[tup]['left'][left] = 0
            if right not in mutual_info[tup]['right']:
                mutual_info[tup]['right'][right] = 0
            mutual_info[tup]['left'][left] += 1
            mutual_info[tup]['right'][right] += 1
    
    
        words = []
        for word in result.keys():
            if word in mutual_info:
                left = np.array(list(mutual_info[word]['left'].values()))
                right = np.array(list(mutual_info[word]['right'].values()))
                ls = info_entropy(left)
                rs = info_entropy(right)
                ms = ls+rs
                words.append(word)
                result[word]['entrophy'] = ms
                result[word]['score'] = result[word]['entrophy']+result[word]['ami']
            if word not in mutual_info:
                result[word]['entrophy'] = 0
                result[word]['score'] = 0

        for key,values in result.items():
            #finalresult.append([key,values['pmi'],values['ami'],values['entrophy'], values['freq'] , values['score']])
        #df = pd.DataFrame(finalresult,columns = ['key','pmi','ami','entrophy','freq','score'])
        #df = df[(df['entrophy']>1)&(df['pmi']>3)&(df['ami']>0.5)&(df['score']>2)]
        #df0 = df0.append(df)
        #发现此处dataframe合并的操作非常费时，改为直接输出词
            if values['pmi']>3 and values['ami']>0.5 and values['entrophy']>1 and values['score']>2:#此处阈值是根据一个500M的样本粗略设定的
                f = open('/mnt/disk1/jiaoyimin/result_1/'+txt_fn.split('/')[-1],'a+')
                if (key[0] not in stopwords1) and (key[1] not in stopwords1) and (not key[0].isdigit()) and (not key[1].isdigit()):#输出。若词组构成部分有停用词，删去
                    if (not key[0].isspace()) and (not key[1].isspace()):
                        f.write(key[0]+key[1]+'\r\n')
                    
    #with open('/mnt/disk1/jiaoyimin/result_1/'+txt_fn.split('/')[-1],'r') as f:
       #x = f.read().splitlines()
    #dict0=dict(Counter(list(x)))#合并相同词并重新输出
    #f =  open('/mnt/disk1/jiaoyimin/result1_1/'+txt_fn.split('/')[-1],'a+') 
    #for key in dict0:
        #if ContainChinese(word):
            #f.write(key+'\r\n')




  0%|          | 0/93 [00:00<?, ?it/s]

/mnt/disk1/jiaoyimin/fenlei/中国医学.txt Cut words
/mnt/disk1/jiaoyimin/fenlei/中国医学.txt Done Cut



  1%|          | 1/93 [1:20:20<123:12:09, 4820.98s/it]

/mnt/disk1/jiaoyimin/fenlei/大学学报(工业技术).txt Cut words
/mnt/disk1/jiaoyimin/fenlei/大学学报(工业技术).txt Done Cut



  2%|▏         | 2/93 [2:26:06<115:13:18, 4558.23s/it]

...


 98%|█████████▊| 91/93 [49:46:33<1:55:23, 3461.89s/it]

/mnt/disk1/jiaoyimin/fenlei/大学学报(自然科学).txt Cut words
/mnt/disk1/jiaoyimin/fenlei/大学学报(自然科学).txt Done Cut



 99%|█████████▉| 92/93 [51:01:38<1:02:54, 3774.79s/it]

/mnt/disk1/jiaoyimin/fenlei/环境科学与安全科学.txt Cut words
/mnt/disk1/jiaoyimin/fenlei/环境科学与安全科学.txt Done Cut



100%|██████████| 93/93 [51:36:47<00:00, 1997.93s/it]  


In [3]:
task_list = glob.glob('/mnt/disk1/jiaoyimin/result_1/*.txt')

In [7]:
for txt_file in tqdm(task_list):
    f = open(txt_file,'r')
    x = f.read().splitlines()
    dict0=dict(Counter(list(x)))#合并相同词并重新输出
    file =  open('/mnt/disk1/jiaoyimin/result_1_1/'+txt_file.split('/')[-1],'a+')
    for word in dict0:
        if ContainChinese(word):
            file.write(word+'\r\n')
            

100%|██████████| 93/93 [00:21<00:00,  4.38it/s]


In [41]:
#如果医药卫生总论.txt内存不够
    file = open('/mnt/disk1/jiaoyimin/fenlei/医药卫生总论.txt','r')
    for line in tqdm(file):
        word_list = [x for x in jieba.cut(line.strip(), cut_all=False) if x not in stopwords]     
        tuples4 = generate_ntuple(word_list, 4)
        tuple4 = dict(Counter(list(tuples4)))
    
        all_2tuple = dict()
        all_3tuple = dict()
        all_1tuple = dict()
        all_4tuple = dict()
        for key,value in tuple4.items():
            if len(key)==2:
                all_2tuple[key]=value
            if len(key)==3:
                all_3tuple[key]=value
            if len(key)==1:
                all_1tuple[key]=value  
            if len(key)==4:
                all_4tuple[key]=value
    
    
    ###all_tuples.append(all_2tuple)
    #all_tuples.append(all_3tuple)
    #all_tuples.append(all_4tuple)
        total2 = len(all_2tuple)
    
        result = defaultdict(dict)
    #pmi = dict()


        for word in all_2tuple:
            left0 = word[:1]
            left_count0 = all_1tuple[left0]
            right0 = word[1:]
            right_count0 = all_1tuple[right0]
            a0 = math.log(total2*all_2tuple[word] / left_count0 / right_count0)
            
        #remain.append(word)
            result[word]['pmi'] = a0
            result[word]['ami'] = a0/(len(word[0])+len(word[1]))
            result[word]['freq'] = all_2tuple[word]
        #pmi[word] = a0

    
        final_words = []

        mutual_info = {}


        for word in all_4tuple:
            left = word[0]
            right = word[3]
            tup = word[1:3]
            if tup not in mutual_info:
                mutual_info[tup] = {'left':{},'right':{}}
            if left not in mutual_info[tup]['left']:
                mutual_info[tup]['left'][left] = 0
            if right not in mutual_info[tup]['right']:
                mutual_info[tup]['right'][right] = 0
            mutual_info[tup]['left'][left] += 1
            mutual_info[tup]['right'][right] += 1
    
    
        words = []
        for word in result.keys():
            if word in mutual_info:
                left = np.array(list(mutual_info[word]['left'].values()))
                right = np.array(list(mutual_info[word]['right'].values()))
                ls = info_entropy(left)
                rs = info_entropy(right)
                ms = ls+rs
                words.append(word)
                result[word]['entrophy'] = ms
                result[word]['score'] = result[word]['entrophy']+result[word]['ami']
            if word not in mutual_info:
                result[word]['entrophy'] = 0
                result[word]['score'] = 0
        final_words = final_words + words
        finalresult = []
        for key,values in result.items():
            #finalresult.append([key,values['pmi'],values['ami'],values['entrophy'], values['freq'] , values['score']])
        #df = pd.DataFrame(finalresult,columns = ['key','pmi','ami','entrophy','freq','score'])
        #df = df[(df['entrophy']>1)&(df['pmi']>3)&(df['ami']>0.5)&(df['score']>2)]
        #df0 = df0.append(df)
        #发现此处dataframe合并的操作非常费时，改为直接输出词，但这样没法看分数了：
            if values['pmi']>3 and values['ami']>0.5 and values['entrophy']>1 and values['score']>2:#此处阈值是根据一个500M的样本粗略设定的
                f = open('/mnt/disk1/jiaoyimin/result/医药卫生总论.txt','a+')
                if (key[0] not in stopwords1) and (key[1] not in stopwords1) and (not key[0].isdigit()) and (not key[1].isdigit()):#输出。若词组构成部分有停用词，删去（问题：非个位数字删不了）
                    f.write(key[0]+key[1]+'\r\n')
   
    with open('/mnt/disk1/jiaoyimin/result/医药卫生总论.txt','r') as f:
        x = f.read().splitlines()
    dict0=dict(Counter(list(x)))#合并相同词并重新输出
    f =  open('/mnt/disk1/jiaoyimin/result1/医药卫生总论.txt','a+') 
    for key in dict0:
        if ContainChinese(word):
            f.write(key+'\r\n')


1749207it [5:03:16, 96.13it/s] 
